IMPORTING 

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

LOADING

In [17]:
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df[['v1', 'v2']]  
df.columns = ['label', 'message'] 

EDA

In [18]:
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:\n", df.head())
print("\nClass Distribution:\n", df['label'].value_counts())

Dataset Shape: (5572, 2)

First 5 Rows:
   label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

Class Distribution:
 label
ham     4825
spam     747
Name: count, dtype: int64


CONVERSION

In [19]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

PREPROCESSING

In [20]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

df['cleaned_message'] = df['message'].apply(clean_text)

TF-IDF

In [21]:
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X = tfidf.fit_transform(df['cleaned_message'])
y = df['label']


MODEL

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

EVALUATION

In [23]:
print("\n=== Model Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


=== Model Evaluation ===
Accuracy: 0.968609865470852

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.77      0.87       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.93      1115
weighted avg       0.97      0.97      0.97      1115


Confusion Matrix:
 [[965   0]
 [ 35 115]]


TEST PREDICTION

In [24]:
def predict_message(text):
    text_cleaned = clean_text(text)
    vect_text = tfidf.transform([text_cleaned])
    prediction = model.predict(vect_text)
    return "Spam" if prediction[0] == 1 else "Not Spam"

sample_text = "Congratulations! You've won a free ticket. Reply CLAIM to get it now."
print("\nSample Message Prediction:", predict_message(sample_text))


Sample Message Prediction: Spam
